In [3]:
import json
import pandas as pd
from coinmarketcap import Market
# function that checks for NaN in a pandas df
def check_nan(df):
    count = 0
    for i,j in enumerate(df.isnull().sum()):
        if j != 0:
            print(df.columns[i], "contains {} NaN".format(j))
            count += 1
    if count == 0:
        print("no NaN")


coinmarketcap = Market()
# the output of below call is byte so we need to convert it to str
a = coinmarketcap.ticker()
# conversion bytecode to str
json_thing = a.decode()
json_thing = json.loads(json_thing)
df = pd.DataFrame(json_thing)
df.last_updated = pd.to_datetime(df.last_updated, unit='s')
del df['available_supply']
del df['id']
del df['percent_change_1h']
del df['percent_change_7d']
del df['name']
del df['total_supply']

df = df[['24h_volume_usd', 'symbol', 'last_updated', 'market_cap_usd',
       'percent_change_24h', 'price_btc', 'price_usd', 'rank']]
check_nan(df)
df.dropna(inplace=True)
check_nan(df)
# df.sort_values('24h_volume_usd', ascending=False, inplace=True)
df

24h_volume_usd contains 52 NaN
last_updated contains 1 NaN
market_cap_usd contains 179 NaN
percent_change_24h contains 59 NaN
price_btc contains 1 NaN
price_usd contains 1 NaN
no NaN


,24h_volume_usd,symbol,last_updated,market_cap_usd,percent_change_24h,price_btc,price_usd,rank
0,1942720000.0,BTC,2017-05-22 17:14:11,37173958266.0,10.77,1.0,2274.32,1
1,1174320000.0,ETH,2017-05-22 17:14:26,16814072192.0,24.96,0.0824709,183.096,2
2,230740000.0,XRP,2017-05-22 17:14:17,12241402045.0,-5.68,0.0001431,0.31769,3
3,34578600.0,XEM,2017-05-22 17:14:24,2212497000.0,-1.55,0.00011073,0.245833,4
4,172184000.0,LTC,2017-05-22 17:14:01,1357259150.0,0.02,0.0119356,26.4986,5
5,69912600.0,DASH,2017-05-22 17:14:07,1013204048.0,37.54,0.0624176,138.575,6
6,181074000.0,ETC,2017-05-22 17:14:38,915593830.0,33.39,0.00448939,9.96702,7
7,55541800.0,BCN,2017-05-22 17:14:13,627292007.0,-7.52,0.00000154,0.00342924,8
8,26080900.0,XMR,2017-05-22 17:14:11,591458164.0,20.23,0.0183682,40.7797,9
9,93039400.0,XLM,2017-05-22 17:14:16,525445468.0,0.88,0.00002459,0.0545921,10
